In [1]:
# Install dependencies
!pip install -q unsloth transformers qwen-vl-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 10.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 87.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.4/170.4 MB 1.9 MB/

In [2]:
# 1. Load libraries
import pandas as pd
import numpy as np
import torch
from PIL import Image
from unsloth import FastVisionModel
from transformers import AutoProcessor
from qwen_vl_utils import process_vision_info
from sklearn. metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-03 20:08:07.780231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764792487.972779      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764792488.027997      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from unsloth import FastVisionModel
from transformers import AutoProcessor
import torch

model_name = "Qwen/Qwen2.5-VL-7B-Instruct"

model, tokenizer = FastVisionModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=False,     # disable 4-bit
    load_in_8bit=False,     # disable 8-bit
    dtype=torch.float16,    # <-- NEW correct argument
    max_seq_length=2048,
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(model_name)

print("✓ Qwen2.5-VL-7B-Instruct loaded in FP16!")

==((====))==  Unsloth 2025.11.6: Fast Qwen2_5_Vl patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

In [ ]:
classify_prompt = """
You are an expert in understanding memes, political symbolism, and image-based social communication.

You are given:
1. An image.
2. A detailed visual description of the image created by an expert.

Your task is to determine whether the meme contains political content based on BOTH the image and its description.

Classify the meme into ONE of the following categories:

1. **Political**
   The meme directly or indirectly references:
   - politicians or political leaders
   - political parties, ideologies, movements, activism
   - government actions, public policies, elections
   - national or international political events
   - political satire, propaganda, criticism, or commentary
   - symbols, slogans, colors, or imagery strongly associated with political groups


2. **NonPolitical**
   The meme does NOT contain political meaning.  
   Examples include:
   - general humor or relatable content
   - personal jokes, daily-life scenarios
   - lifestyle topics

**Important instructions:**
- Use the description to understand hidden or symbolic meaning.
- Consider BOTH explicit and implicit political cues.
- If the text or content is related to Bangla or Bangladeshi politics in any way, classify it as Political.
- Sarcasm, satire, indirect jokes, and symbolic references still count as Political **if** related to politics.
- Be strict: classify only if political meaning is present.

**Respond using ONLY ONE WORD:**  
**Political** or **NonPolitical**
"""

In [ ]:
desc_prompt = """
You are an advanced visual analyst. Carefully observe the image and describe it in clear detail.

Provide a structured description covering the following:

1. **Objects and People**  
   - Identify all visible people, animals, objects, scenes, backgrounds, signs, banners, logos, or notable symbols.

2. **Text in the Image**   
   - Transcribe ALL readable text EXACTLY as it appears (including memes, captions, labels, posters, banners, screenshots, etc.)  
   - If text is unclear, describe what can be visually inferred.

3. **Actions and Context**  
   - Describe what is happening in the image.  
   - Mention any gestures, expressions, settings (e.g., rally, street, office).

4. **Symbols and Implied Meaning**  
   - Mention any noticeable symbols, colors, flags, signs, or elements that could indicate identity, themes, or context.  
   - Describe does it contain information about political parties, logos, leaders, nomination, election, islamic symbol

**Important:**   
- Only describe what you see and read in the image as accurately as possible.

Return the description as details as possible that represent undelying context of the meme.
"""

print("OKK")

In [ ]:
def describe_image(image_path, desc_prompt):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": desc_prompt}
            ]
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    image = Image.open(image_path).convert("RGB")

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=450,
        temperature=0.2,
    )

    # Only keep *generated* tokens, not the whole prompt
    input_len = inputs["input_ids"].shape[1]
    generated_tokens = outputs[:, input_len:]

    description = processor.batch_decode(
        generated_tokens,
        skip_special_tokens=True
    )[0].strip()

    return description


In [ ]:
def classify_with_description(image_path, description, classify_prompt):
    full_prompt = (
        classify_prompt
        + "\n\nHere is an image description created by a visual expert:\n\n"
        + description
        + "\n\nNow analyze BOTH the image and the description carefully."
        + "\nRespond with ONLY ONE word: Political or NonPolitical."
    )


    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": full_prompt}
            ]
        }
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.1,
    )

    result = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    res = result.lower().strip().split()[-1].strip(".,!?\"'")
    
    # Exact match first
    if res in ["political", "politics"]:
        return "Political"
    elif res in ["nonpolitical", "non-political", "not political", "not-political"]:
        return "NonPolitical"

In [ ]:
def predict_image_two_stage(image_path):
    # Stage 1: Describe
    description = describe_image(image_path, desc_prompt)

    # print(description)

    # Stage 2: Classify with description
    prediction = classify_with_description(image_path, description, classify_prompt)

    # print(prediction)

    return prediction, description

In [ ]:
# Load test CSV
test_csv_path = "/kaggle/input/polimem/PoliMemeDecode/Test/Test.csv"
test_images_dir = "/kaggle/input/polimem/PoliMemeDecode/Test/Image"

df_test = pd.read_csv(test_csv_path)
df_test = df_test.head(4)
print(f"Total test samples: {len(df_test)}")

In [ ]:
test_preds = []
descriptions = []   # optional: store descriptions

for _, row in tqdm(df_test.iterrows(), total=len(df_test), desc="2-Stage Predicting"):
    image_filename = row['Image_name']
    image_path = os.path.join(test_images_dir, image_filename)

    if not os.path.exists(image_path):
        test_preds.append("NonPolitical")
        descriptions.append("Image missing")
        continue

    pred, desc = predict_image_two_stage(image_path)
    test_preds.append(pred)
    descriptions.append(desc)


In [ ]:
submission = pd.DataFrame({
    'Image_name': df_test['Image_name'],
    'Label': test_preds
})

submission.to_csv("submission.csv", index=False)

# Optional: descriptions for debugging
desc_df = pd.DataFrame({
    'Image_name': df_test['Image_name'],
    'Description': descriptions,
    'Prediction': test_preds,
})
desc_df.to_csv("debug_descriptions.csv", index=False)

print("✓ Saved submission.csv and debug_descriptions.csv")